# Collecting Weather Climate Data for period 2019

In this section, we collect the Weather Climate Data for the period of 2019. The data collected will act as part of the exogenous variables for the SARIMA forecast of the dengue cases for the 'unseen' 2019.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re
import requests 
import pandas as pd
import pickle

In [2]:
driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')

C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\3074833328.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')


In [3]:
def get_html_table_as_df(table_xpath,df_to_save):
    '''This saves a html table as a df. 
    Rmb to create an empty df to save the data to before executing this function. 
    If the df to save the data to is not empty, this function appends cases'''
      
    # Use find_element_by_xpath method to locate an element
    table_html = driver.find_element(by=By.XPATH, value=table_xpath).get_attribute('outerHTML')
    
    data_table = pd.read_html(table_html)
    df = data_table[0]
    if df_to_save.empty:
        df_to_save = data_table[0]
    else: df_to_save = df_to_save.append(data_table[0])
    return df_to_save

In [4]:
mss_url = 'http://www.weather.gov.sg/climate-historical-daily/'
driver.get(mss_url)
time.sleep(2)

In [5]:
def button_dropdown_option_loop(option_xpath_id, number_of_options, display_button_xpath, 
                                table_xpath, df_to_save):
    '''This function selects the button and clicks on each option in order, starting from the top.
    Diplay button is then clicked to display the table below, and saves the table as a df'''
    for option in range(1,number_of_options+1):
        driver.find_element(by=By.XPATH, value=option_xpath_id).click()
        driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option)+']/a').click()
        driver.find_element(by=By.XPATH, value=display_button_xpath).click()
        time.sleep(2)
        df_to_save = get_html_table_as_df(table_xpath, df_to_save) 
        time.sleep(1)
    return df_to_save

In [6]:
def button_dropdown_option(option_xpath_id, option_index, display_button_xpath):
    '''This function selects the button and clicks on a specific option.
    Index[1] is the first option from the top. 
    Diplay button is then clicked to display the table below'''
    driver.find_element(by=By.XPATH, value=option_xpath_id).click()
    driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option_index)+']/a').click()
    driver.find_element(by=By.XPATH, value=display_button_xpath).click()
    time.sleep(1)

In [7]:
month_option_xpath_id = '//*[@id="monthDiv"]'
display_button_xpath = '//*[@id="display"]'
year_option_xpath_id = '//*[@id="yearDiv"]'

table_xpath = '//*[@id="temp"]/h4[2]/div/div/table'

### we are gonna do this one at a time because:
### a) this page loads very slowly
### b) we need to add a year column
### c) random errors seem to be happening (but seem to have disappeared when I increased the timer)
### d) I don't know how to create a loop that creates variabe names dynamically (e.)

In [8]:
##2019 
df_temp_rain2019 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,2,display_button_xpath)
df_temp_rain2019 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2019)
df_temp_rain2019['Year'] = 2019

C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append(data_table[0])
C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\1454306140.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  else: df_to_save = df_to_save.append

In [9]:
df_temp_rain2019.head()


,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year
0,1 Dec,16.4,10.0,13.2,15.2,26.4,31.0,23.4,6.6,31.5,2019
1,2 Dec,7.2,5.8,6.2,6.8,26.5,30.2,24.0,6.9,29.6,2019
2,3 Dec,3.0,2.4,2.6,2.6,27.4,31.7,24.2,6.0,31.5,2019
3,4 Dec,15.2,9.2,9.2,9.2,27.1,30.3,24.5,5.2,31.5,2019
4,5 Dec,8.2,7.0,8.0,8.0,26.8,30.9,25.2,6.7,24.1,2019


In [10]:
# drop station as it is not relevant
df_temp_rain2019.isnull().sum()

Date                             0
Daily Rainfall Total (mm)        0
Highest 30-min Rainfall (mm)     0
Highest 60-min Rainfall (mm)     0
Highest 120-min Rainfall (mm)    0
Mean Temperature (°C)            0
Maximum Temperature (°C)         0
Minimum Temperature (°C)         0
Mean Wind Speed (km/h)           0
Max Wind Speed (km/h)            0
Year                             0
dtype: int64

In [11]:
df_temp_rain2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 30
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           365 non-null    object 
 1   Daily Rainfall Total (mm)      365 non-null    float64
 2   Highest 30-min Rainfall (mm)   365 non-null    float64
 3   Highest 60-min Rainfall (mm)   365 non-null    float64
 4   Highest 120-min Rainfall (mm)  365 non-null    float64
 5   Mean Temperature (°C)          365 non-null    float64
 6   Maximum Temperature (°C)       365 non-null    float64
 7   Minimum Temperature (°C)       365 non-null    float64
 8   Mean Wind Speed (km/h)         365 non-null    object 
 9   Max Wind Speed (km/h)          365 non-null    float64
 10  Year                           365 non-null    int64  
dtypes: float64(8), int64(1), object(2)
memory usage: 34.2+ KB


In [12]:
df_temp_rain2019 = df_temp_rain2019.replace('-', pd.np.nan)


C:\Users\wongh\AppData\Local\Temp\ipykernel_4176\322834409.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_temp_rain2019 = df_temp_rain2019.replace('-', pd.np.nan)


In [13]:
# Convert Mean Wind Speed to float
df_temp_rain2019['Mean Wind Speed (km/h)'] = df_temp_rain2019['Mean Wind Speed (km/h)'].astype(float)

In [14]:
# impute median values due to values being right-skewed
df_temp_rain2019['Mean Wind Speed (km/h)'].fillna(df_temp_rain2019['Mean Wind Speed (km/h)'].median(), inplace=True)

In [15]:
df_temp_rain2019['Date'].info()

<class 'pandas.core.series.Series'>
Int64Index: 365 entries, 0 to 30
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
365 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [16]:
# Convert the date column into a datetime column
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['Date'] + ' ' + df_temp_rain2019['Year'].astype(str), format='%d %b %Y')

In [17]:
df_temp_rain2019.head(10)

,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year,date
0,1 Dec,16.4,10.0,13.2,15.2,26.4,31.0,23.4,6.6,31.5,2019,2019-12-01
1,2 Dec,7.2,5.8,6.2,6.8,26.5,30.2,24.0,6.9,29.6,2019,2019-12-02
2,3 Dec,3.0,2.4,2.6,2.6,27.4,31.7,24.2,6.0,31.5,2019,2019-12-03
3,4 Dec,15.2,9.2,9.2,9.2,27.1,30.3,24.5,5.2,31.5,2019,2019-12-04
4,5 Dec,8.2,7.0,8.0,8.0,26.8,30.9,25.2,6.7,24.1,2019,2019-12-05
5,6 Dec,0.0,0.0,0.0,0.0,27.1,31.6,24.9,5.6,29.6,2019,2019-12-06
6,7 Dec,5.0,5.0,5.0,5.0,27.6,32.7,25.5,6.3,35.2,2019,2019-12-07
7,8 Dec,22.0,17.2,19.4,20.0,26.6,30.1,24.9,6.9,29.6,2019,2019-12-08
8,9 Dec,0.0,0.0,0.0,0.0,26.6,29.9,24.5,6.5,25.9,2019,2019-12-09
9,10 Dec,0.0,0.0,0.0,0.0,27.2,30.2,24.7,6.2,18.5,2019,2019-12-10


In [18]:
# convert the date column to datetime
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['date'])

# sort the dataframe by date in descending order
df_temp_rain2019 = df_temp_rain2019.sort_values('date', ascending=True)

# reset the index to start at 0
df_temp_rain2019 = df_temp_rain2019.reset_index(drop=True)

In [19]:
df_temp_rain2019.head()

,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year,date
0,1 Jan,25.2,8.8,11.8,11.8,25.7,30.1,23.2,6.9,31.5,2019,2019-01-01
1,2 Jan,50.0,8.2,14.6,20.4,24.3,24.9,23.7,4.2,22.2,2019,2019-01-02
2,3 Jan,6.0,5.2,5.2,5.4,25.9,30.5,23.5,5.2,22.2,2019,2019-01-03
3,4 Jan,4.8,3.4,3.4,3.4,26.4,30.5,24.4,7.3,25.9,2019,2019-01-04
4,5 Jan,1.0,0.6,1.0,1.0,27.7,32.3,25.0,6.7,22.2,2019,2019-01-05


In [20]:
df_temp_rain2019.drop(columns=['Year','Date'], inplace=True)

In [21]:
df_temp_rain2019.head()

,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),date
0,25.2,8.8,11.8,11.8,25.7,30.1,23.2,6.9,31.5,2019-01-01
1,50.0,8.2,14.6,20.4,24.3,24.9,23.7,4.2,22.2,2019-01-02
2,6.0,5.2,5.2,5.4,25.9,30.5,23.5,5.2,22.2,2019-01-03
3,4.8,3.4,3.4,3.4,26.4,30.5,24.4,7.3,25.9,2019-01-04
4,1.0,0.6,1.0,1.0,27.7,32.3,25.0,6.7,22.2,2019-01-05


In [22]:
# Convert the 'date' column to a datetime format
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['date'])

# Set the 'date' column as the index of the DataFrame
df_temp_rain2019 = df_temp_rain2019.set_index('date')

# Resample the DataFrame to a weekly frequency, calculating the mean of the seven daily readings for each week
weekly_df2019 = df_temp_rain2019.resample('W-MON', origin='start').mean().round(2)

# Reset the index of the resulting DataFrame to make the 'date' column a regular column again
weekly_df2019 = weekly_df2019.reset_index()


In [23]:
weekly_df2019.head()

,date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2019-01-07,12.43,3.74,5.14,6.0,26.59,30.31,24.36,7.14,26.97
1,2019-01-14,0.00,0.00,0.00,0.0,27.59,31.40,25.14,11.53,35.99
2,2019-01-21,1.29,0.86,0.89,1.0,27.53,31.56,25.29,11.61,36.23
3,2019-01-28,0.00,0.00,0.00,0.0,27.87,32.64,24.91,9.91,31.74
4,2019-02-04,0.57,0.57,0.60,0.6,28.27,32.99,25.39,11.43,32.27


## Exporting the Weather Climate Data 2019

In [24]:
# Creating temp_rain.csv
weekly_df2019.to_csv('../../assets/weekly_rain2019.csv')